[![Open In Colab](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/badge/open-in-colab.svg)](https://colab.research.google.com/github/crunchdao/quickstarters/blob/master/competitions/mid-one/quickstarters/momentum_attacker/momentum_attacker.ipynb)

![Banner](https://raw.githubusercontent.com/crunchdao/quickstarters/refs/heads/master/competitions/mid-one/assets/banner.webp)

# Momentum Attacker

This notebook demonstrates how to create an `Attacker` described in [attacker.md](https://github.com/microprediction/midone/blob/main/midone/attackers/attacker.md). You may want to glance at this [notebook](../mean_reversion_attacker/mean_reversion_attacker.ipynb) also, if you seek more context or wish to know how these attackers can be used in a new tournament.



## Setup

In [ ]:
%pip install --upgrade midone

In [ ]:
# Get a new token here: https://hub.crunchdao.com/competitions/mid-one/submit/via/notebook

%pip install --upgrade crunch-cli
!crunch setup --notebook mid-one hello --token aaaabbbbccccddddeeeeffff

## Imports

In [2]:
import math
import typing

import pandas
from midone import HORIZON, Attacker, FEWMean, FEWVar
from midone.accounting.pnlutil import add_pnl_summaries, zero_pnl_summary
from tqdm.auto import tqdm

In [ ]:
import crunch

crunch = crunch.load_notebook()

## Creating a Momentum based Attacker

We derive from `Attacker` and use the utilities `FEWMean` and `FEWVar` to track the running exponentially weighted quantities we need.

In [4]:
class MyAttacker(Attacker):

    def __init__(
        self,
        fast_fading_factor=0.1,
        slow_fading_factor=0.01,
        diff_fading_factor=0.001,
        threshold=2,
        burn_in=100,
        **kwargs
    ):
        super().__init__(**kwargs)

        # Track fast expon-weighted moving average
        self.fast_ewa = FEWMean(fading_factor=fast_fading_factor)

        # Track slow expon-weighted moving average
        self.slow_ewa = FEWMean(fading_factor=slow_fading_factor)

        # Tracks mean and var of the difference between the two
        self.diff_var = FEWVar(fading_factor=diff_fading_factor)

        self.threshold = threshold
        self.countdown = burn_in

    def tick(self, x: float):
        # Update the fast expon avg
        self.fast_ewa.tick(x=x)

        # Update the slow expon avg
        self.slow_ewa.tick(x=x)

        fast_minus_slow = self.fast_ewa.get() - self.slow_ewa.get()
        #  Update var of diff
        self.diff_var.tick(x=fast_minus_slow)

        # Soon we'll be warm
        self.countdown -= 1

    def predict(self, horizon=HORIZON) -> float:
        """
        We buy if signal > threshold*(trailing std of signal)
        """

        if self.countdown > 0:
            return 0  # Not warmed up

        fast_minus_slow = self.fast_ewa.get() - self.slow_ewa.get()
        try:
            fast_minus_slow_std = math.sqrt(self.diff_var.get())

            # Create a buy (>0) or sell (<0) decision
            decision = int(fast_minus_slow / (self.threshold * fast_minus_slow_std))

            return decision
        except ArithmeticError:
            return 0

## Run the attacker on mock data

We use `tick_and_predict` from the parent class as this will track profit and loss for us.

In [5]:
# Always reset an attacker
attacker = MyAttacker()

data = [1, 3, 4, 2, 4, 5, 1, 5, 2, 5, 10] * 100
for x in data:
    y = attacker.tick_and_predict(x=x)

## Run the attacker on real data

We reset the attacker every time it encounters a new stream, but track aggregate statistics.

In [ ]:
x_train, x_test = crunch.load_streams()

In [ ]:
total_pnl = []

for stream in tqdm(x_train):
    attacker = MyAttacker()
    pnl = zero_pnl_summary()

    for message in tqdm(stream, leave=False):
        attacker.tick_and_predict(x=message['x'])

    stream_pnl = attacker.pnl.summary()

    pnl = add_pnl_summaries(pnl, stream_pnl)
    pnl.update({
        'profit_per_decision': pnl['total_profit'] / pnl['num_resolved_decisions']
    })

    total_pnl.append(pnl)

total_pnl = pandas.DataFrame(total_pnl)
total_pnl

## CrunchDAO Code Interface

[Submitting to the CrunchDAO platform requires 2 functions, `train` and `infer`.](https://docs.crunchdao.com/competitions/code-interface) Any line that is not in a function or is not an import will be commented when the notebook is processed.

The content of the function is the same as the example, but the train must save the model to be read in infer.

In [9]:
def train():
    """
    We do not recommend using the train function.
    
    Training should be done before running anything in the cloud environment.
    """

    pass  # no train

In [10]:
def infer(
    stream: typing.Iterator[dict],
):
    """
    Please do not modify the infer function, edit the MyAttacker class directly.

    The core of the attacker logic should be implemented through the attacker classes.
    """

    attacker = MyAttacker()
    total_pnl = zero_pnl_summary()

    yield  # mark as ready

    for message in stream:
        yield attacker.tick_and_predict(x=message['x'])

    stream_pnl = attacker.pnl.summary()
    total_pnl = add_pnl_summaries(total_pnl, stream_pnl)

    total_pnl.update({
        'profit_per_decision': total_pnl['total_profit'] / total_pnl['num_resolved_decisions']
    })

    print(total_pnl)

In [ ]:
crunch.test()

print("Download this notebook and submit it to the platform: https://hub.crunchdao.com/competitions/mid-one/submit/via/notebook")